In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras

In [2]:
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('new.csv')

In [4]:
df.head()

,Unnamed: 0,age,is_female,bmi,children,is_smoker,charges,region_southeast,bmi_category_Obese
0,0,-1.440418,1,-0.453160,-0.909234,1,16884.92400,0,0
1,1,-1.511647,0,0.509422,-0.079442,0,1725.55230,1,1
2,2,-0.799350,0,0.383155,1.580143,0,4449.46200,1,1
3,3,-0.443201,0,-1.305052,-0.909234,0,21984.47061,0,0
4,4,-0.514431,0,-0.292456,-0.909234,0,3866.85520,0,0


In [5]:
X = df.drop(columns=['charges', 'Unnamed: 0'], errors='ignore')
y = np.log1p(df['charges']).astype('float32')

# Keep inputs numeric float32 for Keras
X = X.astype('float32')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print('train:', X_train.shape, 'val:', X_val.shape, 'test:', X_test.shape)

train: (855, 7) val: (214, 7) test: (268, 7)


In [7]:
normalizer = layers.Normalization()
normalizer.adapt(X_train)

model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    normalizer,
    layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
    layers.Dropout(0.1),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

In [8]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.Huber(),
    metrics=[keras.metrics.MeanAbsoluteError(name='mae'), keras.metrics.RootMeanSquaredError(name='rmse')]
)

In [9]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6),
]

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=1000,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/1000
27/27 [==============================] - 1s 8ms/step - loss: 7.3306 - mae: 7.8205 - rmse: 7.9430 - val_loss: 5.0765 - val_mae: 5.5662 - val_rmse: 5.6982 - lr: 0.0010
Epoch 2/1000
27/27 [==============================] - 0s 2ms/step - loss: 2.1859 - mae: 2.6360 - rmse: 3.2214 - val_loss: 0.7115 - val_mae: 1.1138 - val_rmse: 1.3749 - lr: 0.0010
Epoch 3/1000
27/27 [==============================] - 0s 2ms/step - loss: 0.8027 - mae: 1.2063 - rmse: 1.5239 - val_loss: 0.4545 - val_mae: 0.8166 - val_rmse: 1.0477 - lr: 0.0010
Epoch 4/1000
27/27 [==============================] - 0s 2ms/step - loss: 0.6401 - mae: 1.0296 - rmse: 1.2942 - val_loss: 0.3697 - val_mae: 0.7143 - val_rmse: 0.9205 - lr: 0.0010
Epoch 5/1000
27/27 [==============================] - 0s 2ms/step - loss: 0.5735 - mae: 0.9559 - rmse: 1.1977 - val_loss: 0.2961 - val_mae: 0.6128 - val_rmse: 0.8182 - lr: 0.0010
Epoch 6/1000
27/27 [==============================] - 0s 2ms/step - loss: 0.5415 - mae: 0.9203 - rmse: 1.

In [10]:
# Evaluate on held-out test set
test = model.evaluate(X_test, y_test, verbose=1)
print(dict(zip(model.metrics_names, test)))

# Convert back to original charges scale for interpretability
pred_log = model.predict(X_test, verbose=0).squeeze()
pred = np.expm1(pred_log)
true = np.expm1(y_test)

mae_charges = np.mean(np.abs(pred - true))
rmse_charges = np.sqrt(np.mean((pred - true) ** 2))
print({'mae_charges': float(mae_charges), 'rmse_charges': float(rmse_charges)})

9/9 [==============================] - 0s 1ms/step - loss: 0.0790 - mae: 0.2308 - rmse: 0.3842
{'loss': 0.07896684110164642, 'mae': 0.23084834218025208, 'rmse': 0.3841770589351654}
{'mae_charges': 3208.410888671875, 'rmse_charges': 5896.1474609375}
